In [ ]:
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# sparkSession = spark --> in the case of EMR
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")
sc.install_pypi_package("boto3")

In [ ]:
from dateutil import parser
import boto3

# counting file sizes aka number of records
counters = {"MO_1510" + str(day): {"raw_data":0,"hour-6-23":0,"hour-6-23-in-sp":0} for day in range(1,32)}

for key in counters.keys():
    traces_raw = spark.read.csv("s3a://mobility-traces-sp/raw-data/" + key + ".csv")
    counters[key]["raw_data"] = traces_raw.count()
    
    traces_hour_6_23 = spark.read.parquet("s3a://mobility-traces-sp/processed-data/records-between-6-23-hour/" + key + "/")
    counters[key]["hour-6-23"] = traces_hour_6_23.count()
    
    traces_hour_6_23_in_sp = spark.read.parquet("s3a://mobility-traces-sp/processed-data/records_between-6-23-only-in-sp-regions/" + key + "/")
    counters[key]["hour-6-23-in-sp"] = traces_hour_6_23_in_sp.count()


csv_out = "day,raw_data_count,hour_6_23_count,hour_6_23_in_sp_count\n"
for key in counters.keys():
    csv_out += key +","+str(counters[key]["raw_data"])+","+str(counters[key]["hour-6-23"])+","+str(counters[key]["hour-6-23-in-sp"])+"\n"
    
s3 = boto3.client('s3')
s3.put_object(Body=bytes(csv_out,"utf-8"), Bucket='mobility-traces-sp', Key='statistics/number-of-records/number-records-per-file.csv')
    